The big problem with the @property built-in is reuse
* the methods it decorates can't be reused for multiple attributes of same class
* They also can't be reused by unrelated classes

In [2]:
class Homework:
    def __init__(self):
        self._grade = 0
    
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not(0<= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._grade = value

In [3]:
galileo = Homework()
galileo.grade = 95

In [5]:
class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
    
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
    
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._checkk_grade(value)
        self._writing_grade = value
        
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value

In [6]:
#the better way to do this in Python is to use a descriptor
class Grade:
    def __get__(self, instance, instance_type):
        pass
    
    def __set__(self, instance, value):
        pass
    

class Exam:
    #class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [7]:
exam1 = Exam()
exam1.writing_grade = 40
# same as
#Exam.__dict__['writing_grade'].__set__(exam1, 40)

#when i retrieve a property
exam1.writing_grade
#same as
# Exam.__dict__['writing_grade'].__get__(exam1, Exam)

In [8]:
#implementing the Grade descriptor
class Grade:
    def __init__(self):
        self._value = 0
    
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._value = value
        

class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


In [10]:
#but accessing these attrubutes on mulriple Exam instances causes unexpected behavior
second_exam = Exam()
second_exam.writing_grade = 75
print(f'Second {second_exam.writing_grade} is right')
print(f'First  {first_exam.writing_grade} is wrong; '
      f'should be 82')


Second 75 is right
First  75 is wrong; should be 82


The problem is that a single Grade instance is shared across all Exam instances for the class attribute writing_grade

* Grade instance for this attribute is constructed once in the program lifetime, when the Exam class is first defined, not each time an Exam instance is created

In [11]:
#this works well, but it leaks memroy
class Grade:
    def __init__(self):
        self._value = {}
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._value.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._value[instance] = value
        


In [12]:
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._value = WeakKeyDictionary()
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._value.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._value[instance] = value

In [15]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    
    
first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print(f'Second {second_exam.writing_grade} is right')
print(f'First  {first_exam.writing_grade} is right')


Second 75 is right
First  82 is right
